In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
path='C:/Users/mehak/OneDrive - University of Delaware - o365/Beheshti, Rahmat - Mehak - Brennan/data/'

In [2]:
data=pd.read_csv(path+'day_intervals/cohort/cohort_non-icu_30_day_readmission.csv.gz', compression='gzip', header=0, index_col=None)

In [3]:
data.head()

,subject_id,hadm_id,admittime,dischtime,label
0,10000032.0,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,1
1,10000032.0,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,1
2,10000826.0,20032235.0,2146-12-05 19:07:00,2146-12-12 16:30:00,1
3,10000826.0,21086876.0,2146-12-18 17:39:00,2146-12-24 19:55:00,1
4,10000883.0,29957930.0,2124-05-14 17:25:00,2124-05-14 21:10:00,1


In [4]:
from datetime import datetime

data['admittime'] = pd.to_datetime(data['admittime'])
data['dischtime'] = pd.to_datetime(data['dischtime'])

In [5]:
data.head()

,subject_id,hadm_id,admittime,dischtime,label
0,10000032.0,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,1
1,10000032.0,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,1
2,10000826.0,20032235.0,2146-12-05 19:07:00,2146-12-12 16:30:00,1
3,10000826.0,21086876.0,2146-12-18 17:39:00,2146-12-24 19:55:00,1
4,10000883.0,29957930.0,2124-05-14 17:25:00,2124-05-14 21:10:00,1


In [6]:
data['los']=pd.to_timedelta(data['dischtime']-data['admittime'],unit='h')
data['los']=data['los'].astype(str)

In [7]:
data[['days', 'dummy','hours']] = data['los'].str.split(' ', -1, expand=True)
data[['hours','min','sec']] = data['hours'].str.split(':', -1, expand=True)
data['los']=pd.to_numeric(data['days'])*24+pd.to_numeric(data['hours'])

In [8]:
data=data.drop(columns=['days', 'dummy','hours','min','sec'])

In [9]:
data.head()

,subject_id,hadm_id,admittime,dischtime,label,los
0,10000032.0,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,1,24
1,10000032.0,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,1,53
2,10000826.0,20032235.0,2146-12-05 19:07:00,2146-12-12 16:30:00,1,165
3,10000826.0,21086876.0,2146-12-18 17:39:00,2146-12-24 19:55:00,1,146
4,10000883.0,29957930.0,2124-05-14 17:25:00,2124-05-14 21:10:00,1,3


In [10]:
data=data[data['los']>0]

In [11]:
data['los'].min()

1

In [12]:
data['los'].max()

11210

In [38]:
data['subject_id'].nunique()

189382

In [13]:
data[data['label']==1]['hadm_id'].nunique()

88015

In [14]:
data[data['label']==0]['hadm_id'].nunique()

359625

In [15]:
data['hadm_id'].nunique()

447640

In [12]:
cond=pd.read_csv(path+'long_format/diag/long_diag_icd10_roots_norm.csv.gz', compression='gzip', header=0, index_col=None)

In [13]:
cond.head()

,subject_id,hadm_id,root,timedelta
0,15734973,20475282,D58,0
1,15734973,20012083,O44,1798
2,15734973,20012083,D58,1798
3,11442057,21518990,O61,0
4,11442057,21518990,O10,0


In [14]:
cond=cond[cond['hadm_id'].isin(data['hadm_id'])]

In [15]:
cond['subject_id'].nunique()

139350

In [16]:
cond['hadm_id'].nunique()

296593

In [17]:
meds=pd.read_csv(path+'long_format/meds/preproc_med_nonproprietary.csv.gz', compression='gzip', header=0, index_col=None)

In [18]:
meds.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,dose_val_rx
0,17868682,22726960,2160-01-07 10:00:00,aspirin,-1 days +22:00:00.000000000,0 days 04:00:00.000000000,81
1,17067646,20845642,2159-02-23 10:00:00,aspirin,0 days 12:49:00.000000000,4 days 01:49:00.000000000,81
2,17067646,25358552,2159-08-08 10:00:00,aspirin,-1 days +13:54:00.000000000,4 days 21:54:00.000000000,81
3,13359788,27483342,2143-11-22 19:00:00,aspirin,0 days 02:59:00.000000000,1 days 02:59:00.000000000,81
4,15346117,20604717,2195-01-21 10:00:00,aspirin,-1 days +13:23:00.000000000,3 days 00:23:00.000000000,81


In [19]:
meds[['start_days', 'dummy','start_hours']] = meds['start_hours_from_admit'].str.split(' ', -1, expand=True)
meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
meds['start_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])

In [20]:
meds[['start_days', 'dummy','start_hours']] = meds['stop_hours_from_admit'].str.split(' ', -1, expand=True)
meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
meds['stop_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])

In [21]:
meds=meds.drop(columns=['start_days', 'dummy','start_hours','min','sec'])

In [22]:
meds['sanity']=meds['stop_time']-meds['start_time']
meds=meds[meds['sanity']>0]
del meds['sanity']

In [23]:
meds.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,dose_val_rx,start_time,stop_time
0,17868682,22726960,2160-01-07 10:00:00,aspirin,-1 days +22:00:00.000000000,0 days 04:00:00.000000000,81,-2,4
1,17067646,20845642,2159-02-23 10:00:00,aspirin,0 days 12:49:00.000000000,4 days 01:49:00.000000000,81,12,97
2,17067646,25358552,2159-08-08 10:00:00,aspirin,-1 days +13:54:00.000000000,4 days 21:54:00.000000000,81,-11,117
3,13359788,27483342,2143-11-22 19:00:00,aspirin,0 days 02:59:00.000000000,1 days 02:59:00.000000000,81,2,26
4,15346117,20604717,2195-01-21 10:00:00,aspirin,-1 days +13:23:00.000000000,3 days 00:23:00.000000000,81,-11,72


In [24]:
meds=meds[meds['hadm_id'].isin(data['hadm_id'])]

In [25]:
meds=pd.merge(meds,data[['hadm_id','los']],on='hadm_id',how='left')

In [26]:
meds['sanity']=meds['los']-meds['start_time']
meds=meds[meds['sanity']>0]
del meds['sanity']
meds.loc[meds['stop_time'] > meds['los'],'stop_time']=meds.loc[meds['stop_time'] > meds['los'],'los']
del meds['los']

In [27]:
meds['subject_id'].nunique()

165369

In [28]:
meds['hadm_id'].nunique()

377946

In [90]:
proc=pd.read_csv(path+'long_format/proc/preproc_proc_icd10.csv.gz', compression='gzip', header=0, index_col=None)

In [91]:
proc[proc['hadm_id']==24799384]

,subject_id,hadm_id,icd_code,chartdate,admittime,proc_time_from_admit
44900,19999043,24799384,10D17ZZ,2164-12-18 00:00:00,2164-12-18 22:42:00,-1 days +01:18:00
44901,19999043,24799384,04LF3DU,2164-12-21 00:00:00,2164-12-18 22:42:00,2 days 01:18:00
44902,19999043,24799384,B41CYZZ,2164-12-21 00:00:00,2164-12-18 22:42:00,2 days 01:18:00
44903,19999043,24799384,0U7C7ZZ,2164-12-18 00:00:00,2164-12-18 22:42:00,-1 days +01:18:00
44904,19999043,24799384,04LE3DT,2164-12-21 00:00:00,2164-12-18 22:42:00,2 days 01:18:00


In [92]:
#proc=proc[proc['proc_time_from_admit']>=0]
proc[['start_days', 'dummy','start_hours']] = proc['proc_time_from_admit'].str.split(' ', -1, expand=True)
proc[['start_hours','min','sec']] = proc['start_hours'].str.split(':', -1, expand=True)
proc['start_time']=pd.to_numeric(proc['start_days'])*24+pd.to_numeric(proc['start_hours'])

In [93]:
proc=proc.drop(columns=['start_days', 'dummy','start_hours','min','sec'])

In [94]:
proc=proc[proc['start_time']>=0]

In [95]:
proc=pd.merge(proc,data[['hadm_id','los']],on='hadm_id',how='left')

In [96]:
proc['sanity']=proc['los']-proc['start_time']
proc=proc[proc['sanity']>0]
del proc['sanity']

In [59]:
proc=proc[proc['hadm_id'].isin(data['hadm_id'])]

In [45]:
proc['subject_id'].nunique()

7590

In [25]:
proc['hadm_id'].nunique()

13555

In [26]:
labs=pd.read_csv(path+'long_format/labs/long_labs_units_cleaned_norm.csv.gz', compression='gzip', header=0, index_col=None)

In [27]:
labs.head()

,subject_id,hadm_id,valuenum,timedelta,itemid
0,10000032,22841357.0,15.7,95,51277
1,10000032,NaN,16.2,121,51274
2,10000032,NaN,32.4,121,51275
3,10000032,NaN,153.0,121,50861
4,10000032,NaN,3.6,121,50862


In [28]:
labs.shape

(53180040, 5)

In [29]:
labs=labs.dropna()

In [66]:
labs.isna().sum()

subject_id    0
hadm_id       0
valuenum      0
timedelta     0
valuenum.1    0
dtype: int64

In [67]:
21636266/53180040

0.4068493743141224

In [30]:
labs=labs[labs['hadm_id'].isin(data['hadm_id'])]

In [31]:
labs['subject_id'].nunique()

114824

In [32]:
labs['hadm_id'].nunique()

239156

In [ ]:
########MAx Min Length

In [153]:
min_length=2
max_length=72

In [154]:
data=data[(data['los']>=min_length) & (data['los']<=max_length)]

In [155]:
########Select time duration needed

In [29]:
include_time=24

In [30]:
data=data[(data['los']>=include_time)]

In [97]:
proc=proc[proc['hadm_id'].isin(data['hadm_id'])]

In [31]:
meds=meds[meds['hadm_id'].isin(data['hadm_id'])]

In [32]:
cond=cond[cond['hadm_id'].isin(data['hadm_id'])]

In [32]:
data['select_time']=data['los']-include_time
data['los']=include_time

In [98]:
proc=pd.merge(proc,data[['hadm_id','select_time']],on='hadm_id',how='left')

In [99]:
proc['start_time']=proc['start_time']-proc['select_time']

In [100]:
proc[proc['hadm_id']==29784107]

,subject_id,hadm_id,icd_code,chartdate,admittime,proc_time_from_admit,start_time,los,select_time
6882,16550115,29784107,0DB83ZX,2126-05-05 00:00:00,2126-05-04 02:05:00,0 days 21:55:00,-495,24.0,516


In [101]:
proc=proc[proc['start_time']>=0]

In [102]:
proc.head()

,subject_id,hadm_id,icd_code,chartdate,admittime,proc_time_from_admit,start_time,los,select_time
147,10157674,28275930,30233S1,2168-11-19 00:00:00,2168-11-18 00:02:00,0 days 23:58:00,10,24.0,13
382,10347477,23290313,3E04305,2134-03-24 00:00:00,2134-03-23 10:57:00,0 days 13:03:00,9,24.0,4
622,10576094,23192014,027134Z,2117-09-01 00:00:00,2117-08-31 17:20:00,0 days 06:40:00,4,24.0,2
623,10576094,23192014,B2011ZZ,2117-09-01 00:00:00,2117-08-31 17:20:00,0 days 06:40:00,4,24.0,2
982,10920925,20792273,0DJ08ZZ,2171-07-18 00:00:00,2171-07-17 16:17:00,0 days 07:43:00,4,24.0,3


In [103]:
proc['start_time'].max()

15

In [33]:
meds=pd.merge(meds,data[['hadm_id','select_time']],on='hadm_id',how='left')
meds['stop_time']=meds['stop_time']-meds['select_time']
meds['start_time']=meds['start_time']-meds['select_time']
meds=meds[meds['stop_time']>=0]
meds.loc[meds.start_time <0, 'start_time']=0

In [34]:
meds['start_time'].max()

23

In [ ]:
labs=pd.merge(labs,data[['hadm_id','select_time']],on='hadm_id',how='left')
labs['start_time']=labs['start_time']-labs['select_time']
labs=labs[labs['start_time']>=0]

In [ ]:
if stay_duration>0:
    cond['select_time']=stay_duration-exclude_time
    proc['select_time']=stay_duration-exclude_time
    meds['select_time']=stay_duration-exclude_time
    labs['select_time']=stay_duration-exclude_time
    cond=cond.loc[:,cond.loc['time'] < cond.loc['select_time']]
    proc=proc.loc[:,proc.loc['time'] < proc.loc['select_time']]
    meds=meds.loc[:,meds.loc['time'] < meds.loc['select_time']]
    labs=labs.loc[:,labs.loc['time'] < labs.loc['select_time']]
else:
    data['select_time']=data['los']-exclude_time
    data['los']=data['los']-exclude_time
    
    cond=pd.merge(cond,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    cond=cond.loc[:,cond.loc['time'] < cond.loc['select_time']]

    proc=pd.merge(proc,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    proc=proc.loc[:,proc.loc['time'] < proc.loc['select_time']]

    meds=pd.merge(meds,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    meds=meds.loc[:,meds.loc['start_time'] < meds.loc['select_time']]

    labs=pd.merge(labs,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    labs=labs.loc[:,labs.loc['time'] < labs.loc['select_time']]
    

In [ ]:
#bucketing

In [35]:
bucket=2

In [37]:
####Create dictionary

In [39]:
hids=meds['hadm_id'].unique()[0:50]

In [40]:
len(hids)

50

In [82]:
dataDic={}
for hid in hids:
    subDic={}
    grp=data[data['hadm_id']==hid]
    #print(grp)
    dataDic[hid]={'Cond':{},'Proc':{},'Med':{},'label':int(grp['label'])}
    #print(condDic)

In [108]:
dataDic

{28275930: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 23290313: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 23192014: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 20792273: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 26470282: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 24593525: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 27905510: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 29220175: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 26031525: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 20675369: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 21862128: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 23202005: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 29626143: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 20048171: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 28290129: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 24441800: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 28303240: {'Cond': {}, 'Proc': {}, 'Med

In [73]:
maxcount=0
for hid in hids:
    grp=cond[cond['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Cond']={'fids':list(grp['root'])}
    #print(condDic)

In [74]:
maxcount

56

In [ ]:
def pad(grp,final):#,labelsDead,labelsSurvived,pt):
    
    grp=grp.sort_values(by=['time'])
    if(non-lab):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
    if(meds):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
    
    if(proc):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            if(grp1.shape[0]==0):
                grp1['val']=0
            else:
                grp1['val']=1     
            grp1['new_time']=new_time
            final=final.append(grp1)
            dataDic[hid]['Proc'][grp['icd_code'][0]]=
            new_time=new_time+1
            
    if(lab):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
      
    return final

In [ ]:
def full_signal(grp):
    grp=grp.sort_values(by=['time'])
    val=[]
    for i in range(1,los,bucket):
        grp1=grp[(grp['time']>=i) & (grp['time']<i+bucket)]
        if(grp1.shape[0]==0):
            val=val.append(0)
        else:
            val=val.append(1)

In [ ]:
maxcount=0
for hid in hids:
    sub=proc[proc['hadm_id']==hid]
    groups=sub.group_by('icd_code')
    for icd_code, grp in groups:
        sig=full_signal(grp)
        dataDic[hid]['Proc'][icd_code]=list(sig)

In [75]:
maxcount=0
for hid in hids:
    grp=proc[proc['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Proc']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Proc']={'fids':list(grp['icd_code'])}
    #print(condDic)

In [76]:
maxcount

40

In [42]:
los=24

In [43]:
meds=meds.sort_values(by=['start_time'])

In [49]:
sample=meds[meds['hadm_id'].isin(hids[0:3])]

In [50]:
sample.shape

(33, 9)

In [51]:
sample['hadm_id'].nunique()

3

In [52]:
sample.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,start_time,stop_time,select_time
14,10000032,29079034,2180-07-23 17:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 04:25:00,2 days 09:25:00,0,24,29
15,10000032,25742920,2180-08-06 03:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 03:16:00,1 days 22:16:00,0,24,18
18,10000032,25742920,2180-08-06 03:00:00,rifaximin,0 days 03:16:00,1 days 22:16:00,0,24,18
17,10000032,29079034,2180-07-23 20:00:00,rifaximin,0 days 07:25:00,2 days 09:25:00,0,24,29
9,10000032,25742920,2180-08-06 03:00:00,raltegravir,0 days 03:16:00,1 days 22:16:00,0,24,18


In [221]:
def full_signal(grp):
    grp=grp.sort_values(by=['start_time'])
    val=[]
    on_med=0
    for i in range(0,los,bucket):
        start=grp[(grp['start_time']>=i) & (grp['start_time']<i+bucket)]
        stop=grp[(grp['stop_time']>=i) & (grp['stop_time']<i+bucket)]
        if(start.shape[0]==0):
            if(on_med==1):
                if(stop.shape[0]==0):
                    on_med=1
                else:
                    on_med=0
        else:
            on_med=1
        val.append(on_med)
    return val

In [110]:
los=24

In [112]:
final_proc=pd.DataFrame()
proc=proc.sort_values(by=['start_time'])

t=0
for i in tqdm(range(0,los,bucket)):            
    ###PROC
    sub_proc=proc[(proc['start_time']>=i) & (proc['start_time']<i+bucket)].groupby(['hadm_id','icd_code']).agg({'subject_id':'max'})
    sub_proc=sub_proc.reset_index()
    sub_proc['start_time']=t
    if final_proc.empty:
        final_proc=sub_proc
    else:    
        final_proc=final_proc.append(sub_proc)

    t=t+1

100%|██████████| 12/12 [00:00<00:00, 214.29it/s]


In [113]:
final_proc.head()

,hadm_id,icd_code,subject_id,start_time
0,21862128.0,0WP9X0Z,11995308,0
1,23112840.0,0BH17EZ,17211314,0
2,24322966.0,B211YZZ,17549711,0
3,28290129.0,0BH17EZ,12553271,0
4,28290129.0,0BJ08ZZ,12553271,0


In [44]:
los=24
bucket=2

In [52]:
meds['dose_val_rx']=meds['dose_val_rx'].apply(pd.to_numeric, errors='coerce')
#meds['dose_val_rx']=meds['dose_val_rx'].fillna(0)

In [53]:
meds['dose_val_rx']

0           81.0
5066688      NaN
5066689    650.0
5066690    650.0
5066694      NaN
           ...  
5401121      2.0
5557017     20.0
4991314    100.0
6184362    100.0
5490288    300.0
Name: dose_val_rx, Length: 3298215, dtype: float64

In [55]:
import numpy as np

In [72]:
final=pd.DataFrame()
t=0
for i in tqdm(range(0,los,bucket)): 
    sub=meds[(meds['start_time']>=i) & (meds['start_time']<i+bucket)].groupby(['hadm_id','nonproprietaryname']).agg({'stop_time':'max','subject_id':'max','dose_val_rx':np.nanmean})
    sub=sub.reset_index()
    sub['start_time']=t
    t=t+1
    sub['stop_time']=sub['stop_time']/bucket
    if final.empty:
        final=sub
    else:
        final=final.append(sub)

100%|██████████| 6/6 [00:03<00:00,  1.75it/s]


In [59]:
final=final.fillna(0)

In [60]:
final.head()

,hadm_id,nonproprietaryname,stop_time,subject_id,dose_val_rx,start_time
0,20000019,acetaminophen,12.0,10467237,0.0,0
1,20000019,aspirin,12.0,10467237,81.0,0
2,20000019,ciprofloxacin hydrochloride,12.0,10467237,500.0,0
3,20000019,dextrose monohydrate,12.0,10467237,12.5,0
4,20000019,heparin sodium,12.0,10467237,5000.0,0


In [131]:
import numpy as np
for hid in tqdm(hids):
    ###PROCS
    df2=proc[proc['hadm_id']==hid]
    #print(df2)
    df2=df2.pivot(index='start_time',columns='icd_code',values='start_time')
    #print(df2)
    #print("==============")
    
    add_indices = pd.Index(range(los)).difference(df2.index)
    #print(add_indices)
    add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
    #print(add_df)
    #print("==============")
    df2=pd.concat([df2, add_df])
    df2=df2.sort_index()
    #df2=df2.ffill()
    df2=df2.fillna(0)
    
    
    df2[df2>0]=1
    #print(df2)
    #print(df2.to_dict(orient="list"))
    #print("==============")
    #print(df2.head())
    dataDic[hid]['Proc']=df2.to_dict(orient="list")

100%|██████████| 60/60 [00:00<00:00, 215.05it/s]


In [132]:
dataDic

{28275930: {'Cond': {},
  'Proc': {'30233S1': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]},
  'Med': {},
  'label': 0},
 23290313: {'Cond': {},
  'Proc': {'3E04305': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]},
  'Med': {},
  'label': 1},
 23192014: {'Cond': {},
  'Proc': {'027134Z': [0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   'B2011ZZ': [0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0

In [94]:
final.head()

,hadm_id,nonproprietaryname,stop_time,subject_id,start_time
0,20000019,acetaminophen,12.0,10467237,0
1,20000019,aspirin,12.0,10467237,0
2,20000019,ciprofloxacin hydrochloride,12.0,10467237,0
3,20000019,dextrose monohydrate,12.0,10467237,0
4,20000019,heparin sodium,12.0,10467237,0


In [95]:
f2=final.groupby(['hadm_id','nonproprietaryname']).size()
f2.groupby('hadm_id').sum().reset_index()[0].max()

37

In [96]:
final.groupby('hadm_id').size().max()

37

In [97]:
final.shape

(1722810, 5)

In [60]:
meds.shape

(1802806, 9)

In [393]:
sample['val']=1

In [63]:
import numpy as np

In [61]:
hids=final['hadm_id'].unique()

In [62]:
len(hids)

224745

In [68]:
los=12#int(los/bucket)

In [69]:
los

12

In [91]:
hids

array([22726960, 25358552, 20604717, 21310555, 21519719, 29891229,
       28539695, 24083628, 25701630, 22156730, 25779938, 27407754,
       24544765, 26687992, 21053230, 22874746, 24648060, 22052120,
       25462002, 29422845, 28073027, 23329757, 23873150, 29543734,
       25852454, 28286953, 20077376, 22852898, 28057895, 28281951,
       22166779, 23594294, 28109241, 28688245, 29576543, 20314911,
       26782860, 28542451, 20488178, 21241451, 23474287, 21363611,
       27482276, 20813910, 22017616, 20568850, 25603722, 21879432,
       23328997, 21678288], dtype=int64)

In [92]:
for hid in tqdm(hids):
    df2=final[final['hadm_id']==hid]
    print(hid)
    val=df2.pivot(index='start_time',columns='nonproprietaryname',values='dose_val_rx')
    df2=df2.pivot(index='start_time',columns='nonproprietaryname',values='stop_time')
    #val=val.fillna(-1)
    #print(df2)
    add_indices = pd.Index(range(los)).difference(df2.index)
    add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
    df2=pd.concat([df2, add_df])
    df2=df2.ffill()
    df2=df2.fillna(0)
    
    val=pd.concat([val, add_df])
    val=val.ffill()
    val=val.fillna(-1)
    #print(df2.columns)
    df2.iloc[:,1:]=df2.iloc[:,0:].sub(df2.index,0)
    #print(df2.shape)
    #print(val.shape)
    #print(df2.head())
    #print(val.head())
    
    df2[df2>0]=1
    df2[df2<0]=0
    #print(val.columns)
    #print("============")
    val.iloc[:,0:]=df2.iloc[:,0:]*val.iloc[:,0:]
    #print(df2.head())
    #print("============")
    #print(val.head())
    dataDic[hid]['Med']['signal']=df2.iloc[:,0:].to_dict(orient="list")
    dataDic[hid]['Med']['val']=val.iloc[:,0:].to_dict(orient="list")
    #print("============")
    ##########COND#########
#     grp=cond[cond['hadm_id']==hid]
#     if(grp.shape[0]==0):
#         dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
#     else:
#         dataDic[hid]['Cond']={'fids':list(grp['root'])}

 14%|█▍        | 7/50 [00:00<00:00, 63.56it/s]

22726960
25358552
20604717
21310555
21519719
29891229
28539695
24083628
25701630
22156730
25779938
27407754


 42%|████▏     | 21/50 [00:00<00:00, 60.25it/s]

24544765
26687992
21053230
22874746
24648060
22052120
25462002
29422845
28073027
23329757
23873150
29543734
25852454


 70%|███████   | 35/50 [00:00<00:00, 63.21it/s]

28286953
20077376
22852898
28057895
28281951
22166779
23594294
28109241
28688245
29576543
20314911
26782860
28542451


100%|██████████| 50/50 [00:00<00:00, 62.57it/s]

20488178
21241451
23474287
21363611
27482276
20813910
22017616
20568850
25603722
21879432
23328997
21678288


In [93]:
dataDic[22726960]

{'Cond': {},
 'Proc': {},
 'Med': {'signal': {'aluminum hydroxide, magnesium hydroxide, and simethicone': [0.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'aspirin': [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   'oxycodone hydrochloride and acetaminophen': [0.0,
    0.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'pantoprazole sodium': [0.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'pneumococcal vaccine polyvalent': [0.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0]},
  'val': {'aluminum hydroxide, magnesium hydroxide, and simethicone': [-0.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0,
    -1.0],
   'aspirin': [81.0, 81.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   'oxycodone h

In [98]:
final[final['hadm_id']==24947999]

,hadm_id,nonproprietaryname,stop_time,subject_id,start_time
716318,24947999,allopurinol,12.0,10000980,0
716319,24947999,aspirin,12.0,10000980,0
716320,24947999,atorvastatin calcium,12.0,10000980,0
716321,24947999,clopidogrel bisulfate,12.0,10000980,0
716322,24947999,dextrose monohydrate,12.0,10000980,0
716323,24947999,furosemide,12.0,10000980,0
716324,24947999,heparin sodium,12.0,10000980,0
716325,24947999,insulin lispro,12.0,10000980,0
716326,24947999,ipratropium bromide,12.0,10000980,0
716327,24947999,magnesium sulfate in water,12.0,10000980,0


In [94]:
d = {'start_time': [1, 2,3], 'itemid': ["med1", "med2","med1"],'rate':[100,50,200],'stop_time': [2, 4,6]}
df2 = pd.DataFrame(data=d)

In [95]:
df2

,start_time,itemid,rate,stop_time
0,1,med1,100,2
1,2,med2,50,4
2,3,med1,200,6


In [97]:
df2.pivot(index='start_time',columns='itemid',values='rate')

itemid,med1,med2
start_time,,
1,100.0,NaN
2,NaN,50.0
3,200.0,NaN


In [85]:
meds[meds['hadm_id']==24947999]

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,start_time,stop_time,select_time
191,10000980,24947999,2190-11-07 12:00:00,aspirin,0 days 15:03:00,1 days 23:03:00,0,24,19
189,10000980,24947999,2190-11-07 14:00:00,insulin lispro,0 days 17:03:00,0 days 20:03:00,0,1,19
195,10000980,24947999,2190-11-07 15:00:00,insulin lispro,0 days 18:03:00,1 days 23:03:00,0,24,19
238,10000980,24947999,2190-11-07 12:00:00,furosemide,0 days 15:03:00,1 days 23:03:00,0,24,19
209,10000980,24947999,2190-11-07 12:00:00,atorvastatin calcium,0 days 15:03:00,1 days 23:03:00,0,24,19
229,10000980,24947999,2190-11-07 14:00:00,dextrose monohydrate,0 days 17:03:00,1 days 23:03:00,0,24,19
224,10000980,24947999,2190-11-07 12:00:00,ipratropium bromide,0 days 15:03:00,1 days 23:03:00,0,24,19
213,10000980,24947999,2190-11-07 14:00:00,magnesium sulfate in water,0 days 17:03:00,1 days 23:03:00,0,24,19
254,10000980,24947999,2190-11-07 12:00:00,allopurinol,0 days 15:03:00,1 days 23:03:00,0,24,19
250,10000980,24947999,2190-11-07 12:00:00,clopidogrel bisulfate,0 days 15:03:00,1 days 23:03:00,0,24,19


In [316]:
sample['start_time'].max()

110

In [ ]:
maxcount=0
for hid in tqdm(hids):
    grp=cond[cond['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Cond']={'fids':list(grp['root'])}

In [223]:
maxcount=0
for hid in tqdm(hids):
    sub=meds[meds['hadm_id']==hid]
    groups=sub.groupby('nonproprietaryname')
    for nonproprietaryname, grp in groups:
        sig=full_signal(grp)
        #print(sig)
        dataDic[hid]['Med'][nonproprietaryname]=list(sig)
    #print(dataDic)

  0%|          | 1008/344107 [02:13<12:34:39,  7.58it/s]


KeyboardInterrupt: 

In [77]:
maxcount=0
for hid in hids:
    grp=meds[meds['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Med']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Med']={'fids':list(grp['nonproprietaryname'])}
    #print(condDic)

In [78]:
maxcount

819

In [ ]:
def full_signal(grp):
    grp=grp.sort_values(by=['start_time'])
    val=[]
    old_lab=np.nan
    for i in range(1,los,bucket):
        start=grp[(grp['start_time']>i) & (grp['start_time']<i+bucket)]
        if(start.shape[0]>0):
            old_lab=start['val'].mean()
        val=val.append(old_lab)
    col_mean = np.nanmean(a, axis=0)
    #Find indices that you need to replace
    inds = np.where(np.isnan(val))

    #Place column means in the indices. Align the arrays using take
    val[inds] = np.take(col_mean, inds[1])

    return val

In [ ]:
maxcount=0
for hid in hids:
    sub=labs[labs['hadm_id']==hid]
    groups=sub.group_by('code')
    for code, grp in groups:
        all_mean=labs[labs['code']==code]['val'].mean()
        subject_mean=labs[(labs['code']==code) & (labs['subject_id'].isin(list(sub['subject_id'])))]['val'].mean()
        sig=full_signal(grp)
        dataDic[hid]['Lab'][code]=list(sig)

In [104]:
cond.groupby('hadm_id').size().max()

56

In [105]:
path='C:/Users/mehak/OneDrive - University of Delaware - o365/Beheshti, Rahmat - Mehak - Brennan/model/data/'

In [106]:
with open(path+'dataDic', 'wb') as fp:
    pickle.dump(dataDic, fp)

In [107]:
with open(path+'hadmDic', 'wb') as fp:
    pickle.dump(hids, fp)

In [108]:
with open(path+'medVocab', 'wb') as fp:
    pickle.dump(list(meds['nonproprietaryname'].unique()), fp)

In [83]:
with open(path+'procVocab', 'wb') as fp:
    pickle.dump(list(proc['icd_code'].unique()), fp)

In [109]:
with open(path+'condVocab', 'wb') as fp:
    pickle.dump(list(cond['root'].unique()), fp)

In [110]:
meds['nonproprietaryname'].nunique()

817

In [86]:
proc['icd_code'].nunique()

4839

In [111]:
cond['root'].nunique()

1517